# Análisis del Código de Entrenamiento de Modelo LSTM para Estimación de Historias de Usuario

Este notebook analiza el código proporcionado para entrenar un modelo LSTM multi-tarea que estima esfuerzo, tiempo y complejidad en historias de usuario basadas en texto (título y Gherkin) y características numéricas extraídas. Como experto en Machine Learning, desglosaré el código, explicaré su funcionalidad, identificaré posibles mejoras y problemas, y proporcionaré ejemplos con un dataset simulado.

**Notas generales sobre el código:**
- Usa PyTorch para el modelo LSTM.
- Incluye preprocesamiento de texto simple (tokenización, vocabulario, padding).
- Ingeniería de características basada en keywords en Gherkin.
- Multi-tarea: regresión para esfuerzo y tiempo, clasificación para complejidad.
- Posibles mejoras: usar embeddings pre-entrenados, validación cruzada, manejo de desbalanceo, etc.

**Dependencias:** Asegúrate de tener instaladas las siguientes librerías:
```bash
pip install pandas torch joblib numpy matplotlib
```

In [268]:
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from collections import Counter
import joblib
from typing import List, Dict
import logging
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_recall_fscore_support
import json
from sklearn.model_selection import KFold
import spacy

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Clase TrainingConfig

Define constantes para el entrenamiento, como rutas, dimensiones del modelo y parámetros de entrenamiento. Es una buena práctica para mantener la configuración centralizada.

**Análisis:**
- **Ventajas:** Facilita la reproducibilidad.
- **Mejoras:** Podría usar un archivo YAML para configs más complejas, o dataclass para tipado estricto.

In [ ]:
class TrainingConfig:
    CSV_PATH = '../stories_dataset.csv'
    MODEL_PATH = 'lstm_model.pth'
    PROCESSOR_PATH = 'text_processor.pkl'
    SCALER_PATH = 'scaler.pkl'
    
    EMBEDDING_DIM = 512
    HIDDEN_DIM = 512  
    BIDIRECTIONAL = True
    DROPOUT = 0.5
    NUM_LAYERS = 3
    
    NUM_EPOCHS = 150
    LEARNING_RATE = 0.00005
    BATCH_SIZE = 16
    PATIENCE = 15
    LOSS_WEIGHTS = {'effort': 1.0, 'time': 2.0}
    
    MAX_SEQ_LEN = 500
    MIN_WORD_FREQ = 1
    
    NUMERICAL_FEATURES = [
        'gherkin_steps', 'gherkin_length', 'num_scenarios', 'num_technical_terms',
        'num_conditions', 'num_entities', 'num_roles',
        'has_frontend', 'has_backend', 'has_security', 'has_payment', 'has_crud',
        'has_reporting', 'has_integration', 'has_notification', 'has_devops_mlops',
        'has_accessibility', 'has_mobile', 'has_testing', 'has_error_handling',
        'has_ui_interaction', 'has_database_query', 'tech_java', 'tech_node',
        'tech_python', 'tech_frontend_framework', 'tech_database', 'tech_infra_cloud'
    ]
    TARGET_COLUMNS = ['effort', 'time']
    TEXT_COLUMNS = ['title', 'gherkin']
    USE_SPACY = True

## Estimation LSTM

Modelo LSTM multi-tarea que toma embeddings de texto concatenados con features numéricas y predice tres salidas.

**Análisis:**
- **Entrada:** Texto tokenizado (secuencia) + features numéricas (repetidas por timestep).
- **Salidas:** Regresión lineal para effort y time; softmax implícito para complejidad (3 clases).
- **Fortalezas:** Multi-tarea eficiente, usa LSTM para capturar secuencias en texto.
- **Debilidades:** Embedding desde cero (no pre-entrenado como GloVe/BERT). LSTM simple (no bidireccional). No dropout para regularización.
- **Mejoras:** Agregar dropout, usar GRU en lugar de LSTM, o transformer para mejor manejo de secuencias largas.

In [270]:
class EstimationLSTM(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, hidden_dim: int, num_features: int, bidirectional: bool, dropout: float, num_layers: int, pretrained_embeddings=None):
        super().__init__()
        if pretrained_embeddings is not None:
            self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
        lstm_dim = embedding_dim
        num_directions = 2 if bidirectional else 1
        self.lstm = nn.LSTM(lstm_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc_effort = nn.Sequential(
            nn.Linear(hidden_dim * num_directions + num_features, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )
        self.fc_time = nn.Sequential(
            nn.Linear(hidden_dim * num_directions + num_features, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )

    def forward(self, text: torch.Tensor, numerical_features: torch.Tensor):
        embedded = self.embedding(text)
        out, _ = self.lstm(embedded)
        last_hidden = out[:, -1, :]
        last_hidden = self.dropout(last_hidden)
        combined = torch.cat((last_hidden, numerical_features), dim=1)
        effort = self.fc_effort(combined)
        time_est = self.fc_time(combined)
        return effort, time_est

## Text Processor

Maneja tokenización, construcción de vocabulario y conversión a secuencias.

**Análisis:**
- Tokenización simple con regex (solo palabras alfanuméricas).
- Vocab con padding y unk.
- **Fortalezas:** Ligero y personalizado.
- **Debilidades:** No maneja stemming/lemmatization, ni subwords. MIN_WORD_FREQ=1 incluye todo (posible ruido).
- **Mejoras:** Usar NLTK/Spacy para tokenización avanzada, o tokenizer de HuggingFace.

In [271]:
class TextProcessor:
    def __init__(self, max_seq_len: int, min_freq: int, use_spacy: bool = False):
        self.vocab = {'<PAD>': 0, '<UNK>': 1}
        self.max_seq_len = max_seq_len
        self.min_freq = min_freq
        self.use_spacy = use_spacy
        self.nlp = spacy.load("es_core_news_sm")

    def _tokenize(self, text: str) -> List[str]:
        if self.use_spacy:
            doc = self.nlp(text.lower())
            return [token.text for token in doc]
        return re.findall(r'\b\w+\.\w+\b|\b\w+\b|[.,!?;]', text.lower())

    def build_vocab(self, texts: pd.Series):
        all_words = []
        for text in texts:
            all_words.extend(self._tokenize(text))
        word_counts = Counter(all_words)
        for word, freq in word_counts.items():
            if freq >= self.min_freq:
                self.vocab[word] = len(self.vocab)
        logging.info(f"Vocabulario construido con {len(self.vocab)} tokens.")
      

    def text_to_sequence(self, text: str) -> List[int]:
        words = self._tokenize(text)
        seq = [self.vocab.get(word, self.vocab['<UNK>']) for word in words]
        if len(seq) > self.max_seq_len:
            seq = seq[:self.max_seq_len]
        else:
            seq += [self.vocab['<PAD>']] * (self.max_seq_len - len(seq))
        return seq

    def save(self, path: str):
        joblib.dump(self, path)

    @staticmethod
    def load(path: str):
        return joblib.load(path)

## Story Dataset

Dataset para PyTorch, combina texto procesado con features numéricas y targets.

**Análisis:**
- Maneja conversión a tensores.
- Incluye fix para NaNs (buena práctica).
- **Debilidades:** Asume 'full_text' existe; complejidad mapeada hardcoded.
- **Mejoras:** Agregar data augmentation para texto (e.g., synonyms).

In [272]:
class StoryDataset(Dataset):
    def __init__(self, df: pd.DataFrame, text_processor: TextProcessor, numerical_cols: List[str], scaler: StandardScaler = None):
        self.df = df
        self.text_processor = text_processor
        self.numerical_cols = numerical_cols
        self.scaler = scaler
        if self.scaler:
            self.df[numerical_cols] = self.scaler.transform(self.df[numerical_cols])

    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> tuple:
        item = self.df.iloc[idx]
        full_text = item['full_text']
        seq_tensor = torch.LongTensor(self.text_processor.text_to_sequence(full_text))
        
        numerical_values = pd.to_numeric(item[self.numerical_cols], errors='coerce').fillna(0).values
        numerical_features = torch.tensor(numerical_values, dtype=torch.float32)
        
        effort = torch.tensor(float(item['effort']), dtype=torch.float32)
        time_est = torch.tensor(float(item['time']), dtype=torch.float32)
        
        return seq_tensor, numerical_features, effort, time_est

## Función extract_features

Ingeniería de características: cuenta steps en Gherkin, flags basados en keywords.

**Análisis:**
- Simple pero efectiva para domain-specific features.
- Usa regex contains (case-insensitive).
- Crea 'complexity' basada en effort (umbral hardcoded).
- **Mejoras:** Usar NLP más avanzado (e.g., TF-IDF, BERT para features), o aprender features end-to-end.

In [273]:
def extract_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica una ingeniería de características robusta, manejando la ausencia
    opcional de la columna 'unit_tests' y detectando tecnologías.
    """
    logging.info("Aplicando ingeniería de características robusta...")
    df_featured = df.copy()

    # --- 1. Validación de Columnas Esenciales y Limpieza ---
    expected_cols = ['id', 'title', 'gherkin', 'effort', 'time']
    if not all(col in df_featured.columns for col in expected_cols):
        missing = [col for col in expected_cols if col not in df_featured.columns]
        raise ValueError(f"Columnas esenciales faltantes en el dataset: {missing}")

    df_featured['gherkin'] = df_featured['gherkin'].astype(str).fillna('')
    df_featured['title'] = df_featured['title'].astype(str).fillna('')

    # --- 2. Creación del Texto Base para Análisis ---
    text_for_keywords = df_featured['title'] + " " + df_featured['gherkin']

    # --- 3. Características Basadas en Categorías y Tecnologías ---
    keyword_categories = {
        'has_frontend': r'frontend|UI|interfaz|CSS|React|Angular|Vue|diseño|vista|pantalla',
        'has_backend': r'backend|servidor|database|base de datos|bd|API|endpoint|SQL|servicios|microservicio|Java|NodeJS|NestJS|Python',
        'has_security': r'seguridad|security|JWT|OAuth|token|autenticación|contraseña|encriptar|CSRF|XSS',
        'has_payment': r'pago|payment|stripe|paypal|tarjeta de crédito|checkout|factura|compra',
        'has_crud': r'crear|añadir|guardar|editar|actualizar|modificar|eliminar|borrar|ver|listar|obtener',
        'has_reporting': r'reporte|dashboard|gráfico|exportar|CSV|PDF|Excel|analíticas|métricas',
        'has_integration': r'api externa|third-party|integración|webhook|sincronizar|CRM|ERP',
        'has_notification': r'notificación|email|correo|SMS|push|alerta|mensaje',
        'has_devops_mlops': r'CI/CD|pipeline|deploy|despliegue|Kubernetes|Docker|monitor|observabilidad|modelo|ML|IA|DevOps',
        'has_accessibility': r'accesibilidad|accessibility|WCAG|lector de pantalla|screen reader|ARIA',
        'has_mobile': r'móvil|app|push|biometría|offline|geolocalización|cámara|gesto',
        'has_testing': r'test|prueba|mock|verificar|validar|assertion|simula',
        'has_error_handling': r'error|excepción|exception|fallo|failure|validar|manejo de error',
        'has_ui_interaction': r'clic|seleccionar|navegar|click|select',
        'has_database_query': r'query|select|sql|database|base de datos|bd'
    }
    for feature_name, pattern in keyword_categories.items():
        df_featured[feature_name] = text_for_keywords.str.contains(pattern, case=False, regex=True, na=False).astype(int)

    tech_stack_keywords = {
        'tech_java': r'java|spring|maven|gradle|JPA|hibernate',
        'tech_node': r'node\.?js|nestjs|express|npm|yarn',
        'tech_python': r'python|django|flask|fastapi|pip',
        'tech_frontend_framework': r'react|angular|vue|svelte',
        'tech_database': r'sql|mysql|postgres|mongodb|redis|base de datos|database',
        'tech_infra_cloud': r'aws|azure|gcp|docker|kubernetes|terraform|S3'
    }
    for tech_name, pattern in tech_stack_keywords.items():
        df_featured[tech_name] = text_for_keywords.str.contains(pattern, case=False, regex=True, na=False).astype(int)

    # --- 4. Características Cuantitativas y Estructurales ---
    gherkin_keywords = [r'\bGiven\b', r'\bWhen\b', r'\bThen\b', r'\bAnd\b', r'\bDado\b', r'\bCuando\b', r'\bEntonces\b', r'\bY\b']
    df_featured['gherkin_steps'] = df_featured['gherkin'].apply(lambda x: sum(len(re.findall(word, x, re.IGNORECASE)) for word in gherkin_keywords))
    df_featured['gherkin_length'] = df_featured['gherkin'].str.len()
    df_featured['num_scenarios'] = df_featured['gherkin'].str.count(r'Scenario:|Escenario:', re.IGNORECASE)
    entities = ['usuario', 'cliente', 'administrador', 'admin', 'sistema', 'desarrollador', 'visitante']
    df_featured['num_entities'] = df_featured['gherkin'].apply(lambda x: sum(x.lower().count(entity) for entity in entities))
    df_featured['num_roles'] = df_featured['gherkin'].str.lower().str.count('admin|user|cliente')
    technical_terms = ['api', 'database', 'authentication', 'frontend', 'backend']
    df_featured['num_technical_terms'] = df_featured['gherkin'].apply(lambda x: sum(x.lower().count(term) for term in technical_terms))
    df_featured['num_conditions'] = df_featured['gherkin'].str.lower().str.count('if|when|si')
    
    # --- 5. Creación del Texto Combinado y Limpieza Final ---
    df_featured['full_text'] = df_featured['title'] + " " + df_featured['gherkin']
    
    if (df_featured['effort'] < 0).any() or (df_featured['time'] < 0).any():
        raise ValueError("Valores negativos encontrados en 'effort' o 'time'")
        
    numerical_cols = TrainingConfig.NUMERICAL_FEATURES
    for col in numerical_cols:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce').fillna(0).astype(float)
    
    logging.info("Ingeniería de características robusta finalizada con éxito.")
    return df_featured

## Función run_training_pipeline

Orquesta todo: carga data, procesa, entrena, guarda. Modificada para usar df de ejemplo y visualizar pérdidas.

**Análisis:**
- Buen flujo end-to-end.
- Usa MSE para regresión, CE para clasificación.
- No hay validación (overfitting posible). Imprime loss cada 5 epochs.
- **Mejoras:** Agregar set de validación, early stopping, métricas (MAE para regresión, accuracy para clasificación). Usar wandb para logging.

Nota: Usamos batch_size=2 y 5 epochs para el ejemplo pequeño.

In [274]:
def evaluate_model(model, loader, device, criterion_regression, weights):
    model.eval()
    total_loss = 0
    effort_loss, time_loss = 0, 0
    all_effort_preds, all_effort_true = [], []
    all_time_preds, all_time_true = [], []
    
    with torch.no_grad():
        for seq, numerical, effort, time_est in loader:
            seq, numerical, effort, time_est = (
                seq.to(device), numerical.to(device), effort.to(device),
                time_est.to(device)
            )
            effort_pred, time_pred = model(seq, numerical)
            
            # Usar view(-1) para asegurar formas compatibles
            loss_effort = criterion_regression(effort_pred.view(-1), effort)
            loss_time = criterion_regression(time_pred.view(-1), time_est)
            loss = (weights['effort'] * loss_effort) + (weights['time'] * loss_time)
            
            total_loss += loss.item()
            effort_loss += loss_effort.item()
            time_loss += loss_time.item()
            
            # Convertir a 1D para evitar problemas con batch size = 1
            effort_pred_1d = torch.atleast_1d(effort_pred.view(-1)).cpu().numpy()
            time_pred_1d = torch.atleast_1d(time_pred.view(-1)).cpu().numpy()
            effort_true_1d = torch.atleast_1d(effort).cpu().numpy()
            time_true_1d = torch.atleast_1d(time_est).cpu().numpy()
            
            all_effort_preds.extend(effort_pred_1d.tolist())
            all_effort_true.extend(effort_true_1d.tolist())
            all_time_preds.extend(time_pred_1d.tolist())
            all_time_true.extend(time_true_1d.tolist())
    
    avg_loss = total_loss / len(loader)
    avg_effort_loss = effort_loss / len(loader)
    avg_time_loss = time_loss / len(loader)
    mae_effort = mean_absolute_error(np.expm1(np.array(all_effort_true)), np.expm1(np.array(all_effort_preds)))
    mae_time = mean_absolute_error(np.expm1(np.array(all_time_true)), np.expm1(np.array(all_time_preds)))
    
    return avg_loss, mae_effort, mae_time, avg_effort_loss, avg_time_loss

In [277]:
def _load_and_prepare_data(config):
    """Carga el dataset desde un CSV, aplica transformaciones y filtros."""
    try:
        df = pd.read_csv(config.CSV_PATH)
        # Transformación logarítmica para reducir el sesgo de valores extremos
        df['effort'] = np.log1p(df['effort'])
        df['time'] = np.log1p(df['time'])
        # Filtrado para eliminar outliers en la escala logarítmica
        df = df[df['time'] <= np.log1p(25)]
        logging.info(f"Dataset filtrado con {len(df)} registros.")
        return df
    except FileNotFoundError:
        logging.error(f"Archivo no encontrado en {config.CSV_PATH}. Abortando.")
        return None

def _perform_training_loop(model, train_loader, val_loader, config, device):
    """Ejecuta el bucle de entrenamiento y validación completo."""
    criterion_regression = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

    history = {'train_losses': [], 'val_losses': [], 'mae_efforts': [], 'mae_times': []}
    best_val_loss = float('inf')
    patience_counter = 0

    logging.info("Iniciando entrenamiento...")
    for epoch in range(config.NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        for seq, numerical, effort, time_est in train_loader:
            # Mover datos al dispositivo
            seq, numerical, effort, time_est = (t.to(device) for t in [seq, numerical, effort, time_est])

            optimizer.zero_grad()
            effort_pred, time_pred = model(seq, numerical)
            
            # Calcular pérdida ponderada
            loss_effort = criterion_regression(effort_pred.squeeze(), effort)
            loss_time = criterion_regression(time_pred.squeeze(), time_est)
            loss = (config.LOSS_WEIGHTS['effort'] * loss_effort) + (config.LOSS_WEIGHTS['time'] * loss_time)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        history['train_losses'].append(avg_train_loss)
        
        # Evaluación del modelo
        val_loss, mae_effort, mae_time, avg_effort_loss, avg_time_loss = evaluate_model(
            model, val_loader, device, criterion_regression, config.LOSS_WEIGHTS
        )
        history['val_losses'].append(val_loss)
        history['mae_efforts'].append(mae_effort)
        history['mae_times'].append(mae_time)
        
        scheduler.step(val_loss)
        
        if (epoch + 1) % 5 == 0:
            logging.info(f"Epoch [{epoch+1}/{config.NUM_EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")
            logging.info(f"MAE Effort: {mae_effort:.4f} horas | MAE Time: {mae_time:.4f} horas")

        # Lógica de Early Stopping y guardado del mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), config.MODEL_PATH)
            logging.info(f"Nuevo mejor modelo guardado con Val Loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= config.PATIENCE:
                logging.info(f"Early stopping activado en la época {epoch+1}.")
                break
    
    return history

def _generate_charts(history):
    """Genera y muestra las configuraciones JSON para las gráficas de Chart.js."""
    epochs = list(range(1, len(history['train_losses']) + 1))
    
    chart_loss_config = {
        "type": "line",
        "data": {
            "labels": epochs,
            "datasets": [
                {"label": "Train Loss", "data": history['train_losses'], "borderColor": "#1f77b4", "fill": False},
                {"label": "Validation Loss", "data": history['val_losses'], "borderColor": "#ff7f0e", "fill": False}
            ]
        },
        "options": {"title": {"display": True, "text": "Curvas de Pérdida (Training y Validation)"}}
    }
    print("```chartjs\n" + json.dumps(chart_loss_config, indent=2) + "\n```")
    
    chart_mae_config = {
        "type": "line",
        "data": {
            "labels": epochs,
            "datasets": [
                {"label": "MAE Effort (horas)", "data": history['mae_efforts'], "borderColor": "#2ca02c", "fill": False},
                {"label": "MAE Time (horas)", "data": history['mae_times'], "borderColor": "#d62728", "fill": False}
            ]
        },
        "options": {"title": {"display": True, "text": "Error Absoluto Medio (MAE)"}}
    }
    print("```chartjs\n" + json.dumps(chart_mae_config, indent=2) + "\n```")

def run_training_pipeline(use_pretrained_embeddings=False):
    """Orquesta el pipeline completo de entrenamiento del modelo."""
    # --- 1. Configuración Inicial ---
    config = TrainingConfig()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"Usando dispositivo: {device}")
    torch.manual_seed(42)

    # --- 2. Carga y Preparación de Datos ---
    df = _load_and_prepare_data(config)
    if df is None:
        return

    # --- 3. Extracción de Features y Preprocesamiento ---
    df_featured = extract_features(df)
    logging.info(f"Distribución 'effort' (original): {np.expm1(df['effort']).describe()}")
    logging.info(f"Distribución 'time' (original): {np.expm1(df['time']).describe()}")

    scaler = StandardScaler()
    scaler.fit(df_featured[config.NUMERICAL_FEATURES])
    
    text_processor = TextProcessor(max_seq_len=config.MAX_SEQ_LEN, min_freq=config.MIN_WORD_FREQ, use_spacy=config.USE_SPACY)
    text_processor.build_vocab(df_featured['full_text'])
    
    # --- 4. Carga de Embeddings Pre-entrenados (Opcional) ---
    pretrained_embeddings = None
    if use_pretrained_embeddings:
        try:
            import fasttext
            model_ft = fasttext.load_model('cc.es.300.bin')
            embedding_matrix = np.zeros((len(text_processor.vocab), config.EMBEDDING_DIM))
            for word, idx in text_processor.vocab.items():
                if word not in ['<PAD>', '<UNK>']:
                    embedding_matrix[idx] = model_ft.get_word_vector(word)[:config.EMBEDDING_DIM]
            pretrained_embeddings = torch.tensor(embedding_matrix, dtype=torch.float32)
            logging.info("Embeddings de FastText cargados correctamente.")
        except (ImportError, FileNotFoundError):
            logging.warning("Módulo 'fasttext' no encontrado o archivo .bin no hallado. Usando embeddings entrenables.")

    # --- 5. Creación de Datasets y DataLoaders ---
    dataset = StoryDataset(df_featured, text_processor, config.NUMERICAL_FEATURES, scaler)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

    # --- 6. Inicialización del Modelo ---
    model = EstimationLSTM(
        vocab_size=len(text_processor.vocab),
        embedding_dim=config.EMBEDDING_DIM,
        hidden_dim=config.HIDDEN_DIM,
        num_features=len(config.NUMERICAL_FEATURES),
        bidirectional=config.BIDIRECTIONAL,
        dropout=config.DROPOUT,
        num_layers=config.NUM_LAYERS,
        pretrained_embeddings=pretrained_embeddings
    ).to(device)

    # --- 7. Entrenamiento ---
    history = _perform_training_loop(model, train_loader, val_loader, config, device)

    # --- 8. Visualización de Resultados ---
    if history:
        _generate_charts(history)

    # --- 9. Guardado de Artefactos Finales ---
    joblib.dump(scaler, config.SCALER_PATH)
    text_processor.save(config.PROCESSOR_PATH)
    logging.info("\n¡Entrenamiento completado!")
    logging.info(f"Modelo guardado en: {config.MODEL_PATH}")
    logging.info(f"Procesador de texto guardado en: {config.PROCESSOR_PATH}")
    logging.info(f"Scaler guardado en: {config.SCALER_PATH}")

# Para ejecutar el pipeline:
run_training_pipeline(False)

2025-10-13 14:20:33,808 - INFO - Usando dispositivo: cpu
2025-10-13 14:20:33,818 - INFO - Dataset filtrado con 324 registros.
2025-10-13 14:20:33,819 - INFO - Aplicando ingeniería de características robusta...
2025-10-13 14:20:33,940 - INFO - Ingeniería de características robusta finalizada con éxito.
2025-10-13 14:20:33,942 - INFO - Distribución 'effort' (original): count    324.000000
mean       8.015432
std        3.288465
min        2.000000
25%        5.000000
50%        8.000000
75%       13.000000
max       13.000000
Name: effort, dtype: float64
2025-10-13 14:20:33,944 - INFO - Distribución 'time' (original): count    324.000000
mean      12.317901
std        5.412477
min        3.000000
25%        8.000000
50%       12.000000
75%       18.000000
max       25.000000
Name: time, dtype: float64
2025-10-13 14:20:36,996 - INFO - Vocabulario construido con 2300 tokens.
2025-10-13 14:20:37,066 - INFO - Iniciando entrenamiento...
2025-10-13 14:21:52,903 - INFO - Nuevo mejor modelo guar

KeyboardInterrupt: 

## Conclusiones y Mejoras Sugeridas

- **Fortalezas:** Código limpio, modular, con logging. Buen uso de PyTorch para multi-tarea.
- **Problemas potenciales:** No maneja datasets grandes (vocab simple), no validación, umbrales hardcoded en complejidad.
- **Mejoras en ML:**
  - Usar cross-validation o train/test split.
  - Embeddings pre-entrenados (e.g., via torchtext).
  - Métricas adicionales: R2 para regresión, F1 para clasificación.
  - Hiperparámetro tuning con Optuna.
  - Manejo de imbalance si complejidad está desbalanceada.
- **Próximos pasos:** Cargar un dataset real y entrenar. Visualizar métricas avanzadas (MAE, accuracy).

Puedes ejecutar este notebook en Jupyter ajustando paths/data. Si necesitas más detalles, ¡pregunta!

In [ ]:
import requests
import gzip
import shutil
import os
from tqdm.auto import tqdm

def descargar_y_descomprimir_fasttext():
    """
    Descarga y descomprime el modelo pre-entrenado de FastText para español.
    """
    # URL directa al modelo de vectores de Common Crawl (cc) para español (es)
    url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz"
    
    # Nombres de los archivos
    archivo_comprimido = "cc.es.300.bin.gz"
    archivo_descomprimido = "cc.es.300.bin"

    print(f"Modelo a descargar: {archivo_comprimido}")
    
    # --- Paso 1: Descargar el archivo con barra de progreso ---
    try:
        # Usamos stream=True para no cargar todo el archivo en memoria
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # Lanza un error si la descarga falla
            
            # Obtener el tamaño total del archivo desde las cabeceras
            total_size_in_bytes = int(r.headers.get('content-length', 0))
            
            # Configurar la barra de progreso de tqdm
            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc="📥 Descargando modelo")
            
            with open(archivo_comprimido, 'wb') as f:
                # Descargar en bloques de 1MB
                for chunk in r.iter_content(chunk_size=1024*1024):
                    progress_bar.update(len(chunk))
                    f.write(chunk)
            progress_bar.close()

        print("\n✅ Descarga completada.")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error durante la descarga: {e}")
        return

    # --- Paso 2: Descomprimir el archivo .gz ---
    print(f"\n⚙️ Descomprimiendo {archivo_comprimido}...")
    try:
        with gzip.open(archivo_comprimido, 'rb') as f_in:
            with open(archivo_descomprimido, 'wb') as f_out:
                # Copiar el contenido descomprimido
                shutil.copyfileobj(f_in, f_out)
        print("✅ Descompresión finalizada.")

    except Exception as e:
        print(f"❌ Error durante la descompresión: {e}")
        return

    # --- Paso 3: Limpiar el archivo comprimido ---
    try:
        print(f"\n🗑️ Eliminando el archivo temporal {archivo_comprimido}...")
        os.remove(archivo_comprimido)
        print("✅ Limpieza completada.")
        print(f"\n✨ ¡Listo! El modelo '{archivo_descomprimido}' está en tu carpeta.")
        
    except OSError as e:
        print(f"❌ Error al eliminar el archivo temporal: {e}")



descargar_y_descomprimir_fasttext()